戸上真人「Pythonで学ぶ音源分離」https://amzn.to/3pRqvII をJuliaでやっていく

# 第3章「音源分離で用いられる数学的知識の基礎」

In [1]:
versioninfo()

Julia Version 1.5.3
Commit 788b2c77c1 (2020-11-09 13:37 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-9.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = "C:\Users\marui\AppData\Local\Programs\Microsoft VS Code\Code.exe"
  JULIA_NUM_THREADS = 


## 第1節 音源分離で用いる線形代数

### 行列の基本的な演算（p.67）

行列（Matrix）は配列（Array）の一種なのでドキュメントは配列のところか、線形代数の標準パッケージを見るといいかも
- https://docs.julialang.org/en/v1/base/arrays
- https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/

他の言語とJuliaとの違いについては公式サイトの次のリンクがうれしい
- https://docs.julialang.org/en/v1/manual/noteworthy-differences/

In [2]:
# 行列を作る
A = [3 1 2
     2 3 1]

# 行列の大きさ・行列の表示
println("size of the matrix: $(size(A))")
println("A = $A")

# スカラー積
c = 2
println("cA = $(c * A)")

# 和
B = [-1 2 4
      1 8 6]
println("A+B = $(A+B)")

# 積
B = [ 4 2
     -1 3
      1 5]
println("AB = $(A*B)")

# アダマール積（要素ごとの積）
B = [-1 2 4
      1 8 6]
println("A*B = $(A.*B)")

# 行列の転置
println("A^T = $(A')")   # 複素数だとエルミート転置（随伴行列）になるので注意
println("A^T = $(transpose(A))")
println("A^T = $(permutedims(A, [2 1]))")   # swapaxesの代わり

# エルミート転置（随伴行列）
A = [3+0im  1+2im  2+3im
     2+0im  3-4im  1+3im]
println("A^H = $(A')")
println("A^H = $(adjoint(A))")

# 行列の積に対するエルミート転置
A = [3+0im  1+2im  2+3im
     2+0im  3-4im  1+3im]
B = [4+4im  2+3im
    -1+1im  3-2im
     1+3im  5+5im]
println("(AB)^H = $((A*B)')")
println("B^H A^H = $(B' * A')")

# 単位行列
using LinearAlgebra
println("I = $(I(3))")
X = Matrix{Float64}(I, 3, 3)   # 型指定をするときにはこういう方法も
println("I = $(X)")

size of the matrix: (2, 3)
A = [3 1 2; 2 3 1]
cA = [6 2 4; 4 6 2]
A+B = [2 3 6; 3 11 7]
AB = [13 19; 6 18]
A*B = [-3 2 8; 2 24 6]
A^T = [3 2; 1 3; 2 1]
A^T = [3 2; 1 3; 2 1]
A^T = [3 2; 1 3; 2 1]
A^H = Complex{Int64}[3 + 0im 2 + 0im; 1 - 2im 3 + 4im; 2 - 3im 1 - 3im]
A^H = Complex{Int64}[3 + 0im 2 + 0im; 1 - 2im 3 + 4im; 2 - 3im 1 - 3im]
(AB)^H = Complex{Int64}[2 - 20im 1 - 21im; 8 - 38im -5 - 8im]
B^H A^H = Complex{Int64}[2 - 20im 1 - 21im; 8 - 38im -5 - 8im]
I = Bool[1 0 0; 0 1 0; 0 0 1]
I = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]


Juliaには`rot180`、`rotl90`、`rotr90`といった行列を回転させる関数も用意されているので、画像処理をするときに便利かもしれない。

アインシュタインの縮約記法（numpyの`einsum`）はEinsum.jlというパッケージを入れれば実現できる。
- https://github.com/ahwillia/Einsum.jl

まずは

```julia
pkg> add Einsum
```

でパッケージをインストールしておく。実行には`@einsum`マクロを使う。結果の行列を新しく作るときには`:=`で、もともとある行列への上書きであれば`=`にする。今回は前者。

In [3]:
using Einsum
A = [3 1 2
     2 3 1]
B = [ 4 2
     -1 3
      1 5]
@einsum C[m, n] := A[m, k] * B[k, n]

println("AB = $(@einsum C[m, n] := A[m, k] * B[k, n])")

AB = [13 19; 6 18]


numpyの`einsum`よりも数式に近くて直感的な気がする。

マクロなので、何が起きているのかを見たければ`@macroexpand`で覗くことができる。（実行結果は省略）

```julia
@macroexpand @einsum C[m, n] := A[m, k] * B[k, n]
```

### アインシュタインの縮約記法を用いたテンソル計算（p.71）

In [4]:
using Random
Random.seed!(0)

L = 10
K = 5
M = 3
R = 3
N = 3

# 行列の準備
A = rand(L, K, M, R) .+ rand(L, K, M, R) * 1im
B = rand(K, R, N) .+ rand(K, R, N) * 1im

# アインシュタインの縮約記法での行列積
@einsum C[l,k,m,n] := A[l,k,m,r] * B[k,r,n]
println("size(C): $(size(C))")

# l=1、k=1について検算
println("A(1,1)B(1,1) = $(A[1,1,:,:] * B[1,:,:])")
println("C(1,1) = $(C[1,1,:,:])")

# 行列積をl、kごとに計算したあと、1方向に和をとる
@einsum C[k,m,n] := A[l,k,m,r] * B[k,r,n]
println("size(C): $(size(C))")

# k=1について検算
C_2 = A[1,1,:,:] * B[1,:,:]
for l in 2:L
    C_2 += A[l,1,:,:] * B[1,:,:]
end
println("C_2(1) = $(C_2)")
println("C(1) = $(C[1,:,:])")

# 縮約記法でのアダマール積
@einsum C[l,k,m,n] := A[l,k,m,n] * B[k,m,n]
println("size(C): $(size(C))")

# 検算
println("A(1,1).*B(1,1) = $(A[1,1,:,:] .* B[1,:,:])")
println("C(1,1) = $(C[1,1,:,:])")

size(C): (10, 5, 3, 3)
A(1,1)B(1,1) = Complex{Float64}[0.08519151565727716 + 1.028382544479421im 0.6841821945018695 + 0.4063384181585944im 0.9468965370847114 + 0.7712941245620732im; -0.4274032034508901 + 1.1323536450340144im 0.13347465647911683 + 1.4375618614587888im 0.6420341784093513 + 0.8057766208887016im; -0.5046213323412478 + 1.3753361976565792im 0.6959288784853086 + 1.1851455333128402im 0.5451961865159336 + 1.545106781292947im]
C(1,1) = Complex{Float64}[0.08519151565727716 + 1.028382544479421im 0.6841821945018695 + 0.4063384181585944im 0.9468965370847114 + 0.7712941245620732im; -0.4274032034508901 + 1.1323536450340144im 0.13347465647911683 + 1.4375618614587888im 0.6420341784093513 + 0.8057766208887016im; -0.5046213323412478 + 1.3753361976565792im 0.6959288784853086 + 1.1851455333128402im 0.5451961865159336 + 1.545106781292947im]
size(C): (5, 3, 3)
C_2(1) = Complex{Float64}[-5.862491672146529 + 9.647132037906514im 0.8803223314751973 + 11.10141687938723im 3.878284908737599 + 11.777

## 第2節 逆行列

## 第3節 ベクトル・行列の微分

## 第4節 確率・統計の基礎知識